# LGBM

Sources
1. [LGBM & Deberta Explained by ZULQARNAIN ALI](https://www.kaggle.com/code/zulqarnainalipk/lgbm-deberta-explained)

In [1]:
import os
import pandas as pd

In [2]:
os.chdir("../")

In [3]:
from lib.config import config
from lib.paths import Paths

## Data Loading

In [4]:
train_df = pd.read_csv(Paths.COMPETITION_TRAIN_CSV_PATH)
test_df = pd.read_csv(Paths.COMPETITION_TEST_CSE_PATH)

## Counting Spelling Errors

In [5]:
import spacy

In [6]:
nlp = spacy.load("en_core_web_sm")

In [7]:
with open(Paths.ENG_WORDS_HX, 'r') as file:
    english_vocab = set(word.strip().lower() for word in file)

In [18]:
def count_spelling_errors(text):
    """Uses `spacy` and list of correctly spelled english words
    located at `Paths.ENG_WORDS_HX` to count number of spelling
    errors.
    """
    doc = nlp(text)
    lemmatized_tokens = [token.lemma_.lower() for token in doc]

    spelling_errors = sum(
        1 for token in lemmatized_tokens if token not in english_vocab
    )

    return spelling_errors


count_spelling_errors("There is one speling error here")

1

## Expanding Contractions

In [9]:
import re
import json

In [12]:
contraction_dict = json.load(open(Paths.CONTRACTION_FILE_PATH, "r"))
contraction_re = re.compile('(%s)' % '|'.join(contraction_dict.keys()))

In [13]:
def expand_contractions(text, c_re=contraction_re):
    """Replaces contracted word/phrase with enlongated word/phrase."""

    def replace(match):
        return contraction_dict[match.group(0)]

    return c_re.sub(replace, text)

expand_contractions("You aren't working!")

'You are not working!'